Perform CIFAR10 classification using Neural Netwroks and Convolutional Neural Networks.

1) Use 10 iterations for training


2) Show the training loss for both networks on the same plot


3) Compare the test loss and accuracy.

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models,transforms
from torchvision.utils import make_grid
from torchvision.datasets import CIFAR10
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary


from torch.utils.data import Dataset, DataLoader




In [79]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [61]:
# CIFAR10
def cifar10(batch_sz, path='./datasets'):
    num_classes = 10
    transform_train = transforms.Compose([
                        transforms.RandomCrop(32, padding=4), # Increase the number of samples by moving the images around 
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                    ])
    transform_test = transforms.Compose([
                        transforms.ToTensor(),
                    ])

    # Training dataset
    train_data = CIFAR10(root=path, train=True, download=True, transform=transform_train) # 
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_sz,
                                               shuffle=True, pin_memory=True)

    # Test dataset
    test_data = CIFAR10(root=path, train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_sz, shuffle=False, pin_memory=True)

    return train_loader, test_loader, num_classes


In [62]:
batch_sz=64 # this is batch size i.e. the number of rows in a batch of data
train_loader, test_loader, _=cifar10(batch_sz) 

tl = iter(train_loader)
batch = next(tl)



Files already downloaded and verified
Files already downloaded and verified


In [64]:
def show_image(batch,img):
    return plt.imshow(np.transpose(batch[0][img,:,:,:],(1,2,0)))

In [116]:
# Define the Class 


# CNN

class CNN(nn.Module):
    def __init__(self, in_channels=3,num_classes=10) -> None:
        super(CNN,self).__init__() # In the other tutorial he is using CNN and self within the super(CNN, self) brakets

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3),stride=(1,1),padding=(1,1))  #  3 -1 /2 ; 3 being the filter_size, with padding the image size stays the same
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=(3,3),stride=(1,1),padding=(1,1)) # this is not useful
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(3,3),stride=(1,1),padding=(1,1))



        self.maxpool = nn.MaxPool2d(kernel_size= (2,2), stride= (2,2))

        self._in = nn.Linear(64*4*4, 1024)
        self.h1 = nn.Linear(1024,512) # remove and adjust dims to recover AC2
        self.h2 = nn.Linear(512,256)
        self.h3 = nn.Linear(256,128)
        self.lin_out = nn.Linear(128,num_classes) # change 128 to 16*7*7 for AC1


    # Forward Pass
    
    def forward(self, x):

        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.maxpool(x)

        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = self.maxpool(x)

        x = torch.relu(self.conv5(x))
        x = torch.relu(self.conv6(x))
        x = self.maxpool(x)

        # Linear Model
        x = x.reshape(x.shape[0], -1)
        x = torch.relu(self._in(x))
        x = torch.relu(self.h1(x))
        x = torch.relu(self.h2(x))
        x = torch.relu(self.h3(x))
        x = self.lin_out(x)
            
        return x

In [95]:
# Dim test

def dim_test(model,dim,channel,x1,x2):
    model = model.to(device=device)
    x = torch.randn(dim,channel,x1,x2)
    return model(x).shape

In [117]:
dim_test(Cifar10CnnModel(),64,3,32,32)

torch.Size([64, 10])

In [109]:
# Hyperparameter

inputs = 3*32*32
mm = 0
batch_sz=64
epoch_no = 3
mini_batches = [64]
learning_rates = [0.05,0.01,0.005]
learning_rates = learning_rates[::-1]

In [119]:
def var_training(model=CNN(),type_CNN = True):
    for mini_batch in mini_batches:
        for learning_rate in learning_rates:
            model_2 = model.to(device=device)
        
            # write to tensorboard
            step = 0

            if type_CNN == True:
                writer = SummaryWriter(f'runs/CIFAR10/CNN_minibatch={mini_batch}_learning_rate={learning_rate}')
            else:
                model = input()
                writer = SummaryWriter(f'runs/CIFAR10/{model}_minibatch={mini_batch}_learning_rate={learning_rate}')


            # Loss and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model_2.parameters(), lr=learning_rate, momentum= mm)
            scheduler = None

            # generate new data
            train_loader, test_loader, _=cifar10(mini_batch) 

            
            
            for epoch in range(epoch_no):
                train_loss = 0
                for batch_idx, (data, targets) in enumerate(train_loader):
                    data = data.to(device=device)
                    targets = targets.to(device=device)

                    if type_CNN == False:
                        data = data.reshape(data.shape[0], -1)
                    else:
                        pass

                    # forwards
                    logits = model_2(data)
                    loss = criterion(logits, targets)
                    train_loss += loss.item()


                    # backward 
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    # scbeduler.step()

                    # Running Accuracy
                    _, predictions = logits.max(1)
                    num_corr = (predictions == targets).sum()
                    running_acc = float(num_corr)/float(data.shape[0])
                    
                    writer.add_scalar("Training Loss", loss, global_step = step) 
                    writer.add_scalar("Accuracy", running_acc, global_step=step)
                    step += 1  

In [118]:
var_training(model=,type_CNN=True)

Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: 

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")